# Road analysis of capital provinces of Veneto

Authors: Biffis Nicola, Pavan Stefano, Trevisi Davide

### Utilities

#### **NOTE**: The cell below is only needed when working in the Google Colab environment

In [ ]:
!pip install osmnx mapclassify

#### Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import osmnx as ox
import networkx as nx
from functools import reduce

pd.set_option('display.max_rows', 8)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 7)

#### Constants

In [ ]:
N = 30
X = 5.0

### Graph creation process

Firstly extract the graph of the intended city from the OSM database with extended boundaries of 500 meters and an intersection tolerance of 15 meters, based on the experimentation, so the bigger intersections are treated as a single node.

In [ ]:
G_Belluno = ox.graph_from_place('Belluno, 32100, Veneto, Italy', network_type="drive", simplify=True)
G_Padova = ox.graph_from_place('Padova, Veneto, Italy', network_type="drive", simplify=True)
G_Rovigo = ox.graph_from_place('Rovigo, 45100, Veneto, Italy', network_type="drive", simplify=True)
G_Treviso = ox.graph_from_place('Treviso, 31100, Veneto, Italy', network_type="drive", simplify=True)
G_Verona = ox.graph_from_place('Verona, Veneto, Italy', network_type="drive", simplify=True)
G_Vicenza = ox.graph_from_place('Vicenza, 36100, Veneto, Italy', network_type="drive", simplify=True)

G_Belluno_proj = ox.project_graph(G_Belluno)
G_Padova_proj = ox.project_graph(G_Padova)
G_Rovigo_proj = ox.project_graph(G_Rovigo)
G_Treviso_proj = ox.project_graph(G_Treviso)
G_Verona_proj = ox.project_graph(G_Verona)
G_Vicenza_proj = ox.project_graph(G_Vicenza)

G_Belluno_final = ox.consolidate_intersections(G_Belluno_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Padova_final = ox.consolidate_intersections(G_Padova_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Rovigo_final = ox.consolidate_intersections(G_Rovigo_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Treviso_final = ox.consolidate_intersections(G_Treviso_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Verona_final = ox.consolidate_intersections(G_Verona_proj , rebuild_graph=True, tolerance=15, dead_ends=False)
G_Vicenza_final = ox.consolidate_intersections(G_Vicenza_proj , rebuild_graph=True, tolerance=15, dead_ends=False)

In [ ]:
ox.save_graphml(G_Belluno_final, "./graphs/belluno.graphml")
ox.save_graphml(G_Padova_final, "./graphs/padova.graphml")
ox.save_graphml(G_Rovigo_final, "./graphs/rovigo.graphml")
ox.save_graphml(G_Treviso_final, "./graphs/treviso.graphml")
ox.save_graphml(G_Verona_final, "./graphs/verona.graphml")
ox.save_graphml(G_Vicenza_final, "./graphs/vicenza.graphml")

## Analysis

In [ ]:
G_Belluno_final = ox.load_graphml("./graphs/belluno.graphml")
G_Padova_final = ox.load_graphml("./graphs/padova.graphml")
G_Rovigo_final = ox.load_graphml("./graphs/rovigo.graphml")
G_Treviso_final = ox.load_graphml("./graphs/treviso.graphml")
G_Verona_final = ox.load_graphml("./graphs/verona.graphml")
G_Vicenza_final = ox.load_graphml("./graphs/vicenza.graphml")

In [ ]:
G = G_Treviso_final

Visualize the generated graph on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
m = edges.explore()
nodes.explore(m=m, marker_kwds={"radius": 4})

Compute all the centralities, sort them by decreasing order and deduplicate the edges

In [ ]:
# Weighted betweenness centrality
weighted_node_bc = nx.betweenness_centrality(G, weight='length')
weighted_edge_bc = nx.edge_betweenness_centrality(G, weight='length')

# Unweighted betweenness centrality
unweighted_node_bc = nx.betweenness_centrality(G, weight=None)
unweighted_edge_bc = nx.edge_betweenness_centrality(G, weight=None)

# Weighted closeness centrality
weighted_node_cc = nx.closeness_centrality(G, distance='length')

# Unweighted closeness centrality
unweighted_node_cc = nx.closeness_centrality(G, distance=None)

nx.set_node_attributes(G, weighted_node_bc, "weighted betweenness centrality")
nx.set_edge_attributes(G, weighted_edge_bc, "weighted betweenness centrality")
nx.set_node_attributes(G, unweighted_node_bc, "unweighted betweenness centrality")
nx.set_edge_attributes(G, unweighted_edge_bc, "unweighted betweenness centrality")
nx.set_node_attributes(G, weighted_node_cc, "weighted closeness centrality")
nx.set_node_attributes(G, unweighted_node_cc, "unweighted closeness centrality")

# Sort by decreasing order
sorted_weighted_node_bc = sorted(weighted_node_bc.items(), key=lambda x: -x[1])
sorted_weighted_edge_bc = sorted(weighted_edge_bc.items(), key=lambda x: -x[1])
sorted_unweighted_node_bc = sorted(unweighted_node_bc.items(), key=lambda x: -x[1])
sorted_unweighted_edge_bc = sorted(unweighted_edge_bc.items(), key=lambda x: -x[1])
sorted_weighted_node_cc = sorted(weighted_node_cc.items(), key=lambda x: -x[1])
sorted_unweighted_node_cc = sorted(unweighted_node_cc.items(), key=lambda x: -x[1])

# Deduplicate the edges
for i in sorted_weighted_edge_bc:
    u = i[0][1]
    v = i[0][0]
    for j in sorted_weighted_edge_bc:
        if j[0][0] == u:
            if j[0][1] == v:
                sorted_weighted_edge_bc.remove(j)

# Deduplicate the edges
for i in sorted_unweighted_edge_bc:
    u = i[0][1]
    v = i[0][0]
    for j in sorted_unweighted_edge_bc:
        if j[0][0] == u:
            if j[0][1] == v:
                sorted_unweighted_edge_bc.remove(j)

Print the weighted betweenness centrality of nodes

In [ ]:
# Print the nodes id and centrality scores
nodes_data = []

for n in sorted_weighted_node_bc:
    nodes_data.append([n[0], n[1]])

sorted_weighted_node_bc_df = pd.DataFrame(nodes_data, columns=["ID", "Weighted Betweenness Centrality"])
display(sorted_weighted_node_bc_df)

Print the unweighted betweenness centrality of nodes

In [ ]:
# Print the nodes id and centrality scores
nodes_data = []

for n in sorted_unweighted_node_bc:
    nodes_data.append([n[0], n[1]])

sorted_unweighted_node_bc_df = pd.DataFrame(nodes_data, columns=["ID", "Unweighted Betweenness Centrality"])
display(sorted_unweighted_node_bc_df)

Print the weighted closeness centrality of nodes

In [ ]:
# Print the nodes id and centrality scores
nodes_data = []

for n in sorted_weighted_node_cc:
    nodes_data.append([n[0], n[1]])

sorted_weighted_node_cc_df = pd.DataFrame(nodes_data, columns=["ID", "Weighted Closeness Centrality"])
display(sorted_weighted_node_cc_df)

Print the unweighted closeness centrality of nodes

In [ ]:
# Print the nodes id and centrality scores
nodes_data = []

for n in sorted_unweighted_node_cc:
    nodes_data.append([n[0], n[1]])

sorted_unweighted_node_cc_df = pd.DataFrame(nodes_data, columns=["ID", "Unweighted Closeness Centrality"])
display(sorted_unweighted_node_cc_df)

Put them all in a single table, sorted by weighted betweenness centrality of nodes

In [ ]:
dfs = [sorted_weighted_node_bc_df, sorted_unweighted_node_bc_df, sorted_weighted_node_cc_df, sorted_unweighted_node_cc_df]

merged_nodes_df = reduce(lambda left, right: pd.merge(left, right, on=['ID'], how='outer'), dfs)
merged_nodes_df.sort_values(by=["Weighted Betweenness Centrality", "Unweighted Betweenness Centrality", "Weighted Closeness Centrality", "Unweighted Closeness Centrality"], inplace=True, ascending = [False, False, False, False])
display(merged_nodes_df)

Print the weighted betweenness centrality of edges

In [ ]:
# Print the edges name and centrality scores
edges_data = []

for e in sorted_weighted_edge_bc:
    if "name" in G.edges[e[0]]:
        name = G.edges[e[0]]["name"]
        if isinstance(name, list):
            edges_data.append([" - ".join(name), e[1]])
        else:
            edges_data.append([G.edges[e[0]]["name"], e[1]])
    elif "ref" in G.edges[e[0]]:
        name = G.edges[e[0]]["ref"]
        if isinstance(name, list):
            edges_data.append([" - ".join([str(n) for n in name]), e[1]])
        else:
            edges_data.append([G.edges[e[0]]["ref"], e[1]])
    else:
        id = G.edges[e[0]]["osmid"]
        if isinstance(id, list):
            edges_data.append([" - ".join([str(n) for n in id]), e[1]])
        else:
            edges_data.append([G.edges[e[0]]["osmid"], e[1]])

sorted_weighted_edge_bc_df = pd.DataFrame(edges_data, columns=["Name", "Weighted Betweenness Centrality Score"])
display(sorted_weighted_edge_bc_df)

Print the unweighted betweenness centrality of edges

In [ ]:
# Print the edges name and centrality scores
edges_data = []

for e in sorted_unweighted_edge_bc:
    if "name" in G.edges[e[0]]:
        name = G.edges[e[0]]["name"]
        if isinstance(name, list):
            edges_data.append([" - ".join(name), e[1]])
        else:
            edges_data.append([G.edges[e[0]]["name"], e[1]])
    elif "ref" in G.edges[e[0]]:
        name = G.edges[e[0]]["ref"]
        if isinstance(name, list):
            edges_data.append([" - ".join([str(n) for n in name]), e[1]])
        else:
            edges_data.append([G.edges[e[0]]["ref"], e[1]])
    else:
        id = G.edges[e[0]]["osmid"]
        if isinstance(id, list):
            edges_data.append([" - ".join([str(n) for n in id]), e[1]])
        else:
            edges_data.append([G.edges[e[0]]["osmid"], e[1]])

sorted_unweighted_edge_bc_df = pd.DataFrame(edges_data, columns=["Name", "Unweighted Betweenness Centrality Score"])
display(sorted_unweighted_edge_bc_df)

Put them all in a single table, sorted by weighted betweenness centrality of edges

In [ ]:
dfs = [sorted_weighted_edge_bc_df, sorted_unweighted_edge_bc_df]

merged_edges_df = reduce(lambda left, right: pd.merge(left, right, on=['Name'], how='outer'), dfs)
merged_edges_df.sort_values(by=["Weighted Betweenness Centrality Score", "Unweighted Betweenness Centrality Score"], inplace=True, ascending = [False, False])
display(merged_edges_df)

In [ ]:
city = ""

if (G == G_Belluno_final):
    city = "Belluno"
elif (G == G_Padova_final):
    city = "Padova"
elif (G == G_Rovigo_final):
    city = "Rovigo"
elif (G == G_Treviso_final):
    city = "Treviso"
elif (G == G_Verona_final):
    city = "Verona"
elif (G == G_Vicenza_final):
    city = "Vicenza"
else:
    city = "ERROR"

Print the plot of the weighted betwenness centrality

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(edges, x='weighted betweenness centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street weighted centrality cumulative distribution')

sns.ecdfplot(nodes, x='weighted betweenness centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions weighted betweenness centrality cumulative distribution')

fig.suptitle('Cumulative weighted betweenness centrality of ' + city, fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-' + city + '.png', dpi=300)

Print the plot of the unweighted betwenness centrality

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(edges, x='unweighted betweenness centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Street unweighted centrality cumulative distribution')

sns.ecdfplot(nodes, x='unweighted betweenness centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions unweighted betweenness centrality cumulative distribution')

fig.suptitle('Cumulative unweighted betweenness centrality of ' + city, fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-' + city + '.png', dpi=300)

Print the plot of the un/weighted closeness centrality

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)

fig, ax = plt.subplots(nrows=2)

sns.ecdfplot(nodes, x='weighted closeness centrality', ax=ax[0])
ax[0].set_xscale('log')
ax[0].set_title('Junctions weighted closeness centrality cumulative distribution')

sns.ecdfplot(nodes, x='unweighted closeness centrality', ax=ax[1])
ax[1].set_xscale('log')
ax[1].set_title('Junctions unweighted closeness centrality cumulative distribution')

fig.suptitle('Cumulative closeness centrality of ' + city, fontsize=16)
fig.set_size_inches(10, 10)
fig.tight_layout()
plt.savefig('figures/cdf-' + city + '.png', dpi=300)

Visualize the weighted betweenness centrality of the edges on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
edges.explore(tiles="cartodbdarkmatter", column="weighted betweenness centrality", cmap="plasma")

Visualize the unweighted betweenness centrality of the edges on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
edges.explore(tiles="cartodbdarkmatter", column="unweighted betweenness centrality", cmap="plasma")

Visualize the weighted betweenness centrality of the nodes on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
nodes.explore(tiles="cartodbdarkmatter", column="weighted betweenness centrality", cmap="plasma")

Visualize the unweighted betweenness centrality of the nodes on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
nodes.explore(tiles="cartodbdarkmatter", column="unweighted betweenness centrality", cmap="plasma")

Visualize the weighted closeness centrality of the nodes on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
nodes.explore(tiles="cartodbdarkmatter", column="weighted closeness centrality", cmap="plasma")

Visualize the unweighted closeness centrality of the nodes on the map

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
nodes.explore(tiles="cartodbdarkmatter", column="unweighted closeness centrality", cmap="plasma")

Visualize the map with top N streets

In [ ]:
nodes = [n[0] for n in sorted_weighted_node_bc[:N]]
edges = [n[0] for n in sorted_weighted_edge_bc[:N]]

Subgraph_nodes = G.subgraph(nodes)
Subgraph_edges = G.edge_subgraph(edges) 

Subgraph_nodes_nodes, Subgraph_nodes_edges = ox.graph_to_gdfs(Subgraph_nodes)
Subgraph_edges_nodes, Subgraph_edges_edges = ox.graph_to_gdfs(Subgraph_edges)

Subgraph_edges_edges.explore(tiles="cartodbdarkmatter", column="weighted betweenness centrality", cmap="plasma")
#Subgraph_nodes_nodes.explore(m=m, tiles="cartodbdarkmatter", column="weighted betweenness centrality", marker_kwds={"radius": 5})

Visualize the map with top Top N intersections overlayed over the top N streets

In [ ]:
nodes = [n[0] for n in sorted_weighted_node_bc[:N]]
edges = [n[0] for n in sorted_weighted_edge_bc[:N]]

Subgraph_nodes = G.subgraph(nodes)
Subgraph_edges = G.edge_subgraph(edges) 

Subgraph_nodes_nodes, Subgraph_nodes_edges = ox.graph_to_gdfs(Subgraph_nodes)
Subgraph_edges_nodes, Subgraph_edges_edges = ox.graph_to_gdfs(Subgraph_edges)

m = Subgraph_edges_edges.explore(tiles="cartodbdarkmatter", color="gray")
Subgraph_nodes_nodes.explore(m=m, tiles="cartodbdarkmatter", column="weighted betweenness centrality", marker_kwds={"radius": 5})

Same but with unweighted centrality

In [ ]:
nodes = [n[0] for n in sorted_weighted_node_bc[:N]]
edges = [n[0] for n in sorted_weighted_edge_bc[:N]]

Subgraph_nodes = G.subgraph(nodes)
Subgraph_edges = G.edge_subgraph(edges) 

Subgraph_nodes_nodes, Subgraph_nodes_edges = ox.graph_to_gdfs(Subgraph_nodes)
Subgraph_edges_nodes, Subgraph_edges_edges = ox.graph_to_gdfs(Subgraph_edges)

m = Subgraph_edges_edges.explore(tiles="cartodbdarkmatter", color="gray")
Subgraph_nodes_nodes.explore(m=m, tiles="cartodbdarkmatter", column="unweighted betweenness centrality", marker_kwds={"radius": 5})

Visualize the map with top X % streets and intersections

In [ ]:
node_top_X_perc = (G.number_of_nodes() * X) / 100.0
edge_top_X_perc = (G.number_of_edges() * X) / 100.0

nodes = [n[0] for n in sorted_weighted_node_bc[:round(node_top_X_perc)]]
edges = [n[0] for n in sorted_weighted_edge_bc[:round(edge_top_X_perc)]]

Subgraph_nodes = G.subgraph(nodes)
Subgraph_edges = G.edge_subgraph(edges)

Subgraph_nodes_nodes, Subgraph_nodes_edges = ox.graph_to_gdfs(Subgraph_nodes)
Subgraph_edges_nodes, Subgraph_edges_edges = ox.graph_to_gdfs(Subgraph_edges)

m = Subgraph_edges_edges.explore(tiles="cartodbdarkmatter", column="weighted betweenness centrality", cmap="plasma")
Subgraph_nodes_nodes.explore(m=m, tiles="cartodbdarkmatter", color="gray", marker_kwds={"radius": 5})

Visualize the map with top X % streets

In [ ]:
edge_top_X_perc = (G.number_of_edges() * X) / 100.0

edges = [n[0] for n in sorted_weighted_edge_bc[:round(edge_top_X_perc)]]

Subgraph_edges = G.edge_subgraph(edges)

Subgraph_edges_nodes, Subgraph_edges_edges = ox.graph_to_gdfs(Subgraph_edges)

Subgraph_edges_edges.explore(tiles="cartodbdarkmatter", column="weighted betweenness centrality", cmap="plasma")